# Exploración de Datasets de OpenML

Este notebook explora y analiza datasets de OpenML para el proyecto de meta-learning.


In [ ]:
import sys
import os
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'src'))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from data_loader import load_openml_dataset, search_openml_datasets
from meta_features import extract_meta_features

# Configuración
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")


## 1. Búsqueda de Datasets


In [ ]:
# Buscar datasets de clasificación
classification_datasets = search_openml_datasets(
    task_type='Supervised Classification',
    n_samples_min=100,
    n_samples_max=5000,
    n_features_max=50
)

print(f"Encontrados {len(classification_datasets)} datasets")
classification_datasets.head()


## 2. Cargar Datasets de Ejemplo


In [ ]:
# Cargar algunos datasets populares de OpenML
# Iris (ID: 61), Wine (ID: 187), Breast Cancer (ID: 13)
dataset_ids = [61, 187, 13]

datasets = []
for dataset_id in dataset_ids:
    print(f"Cargando dataset {dataset_id}...")
    dataset = load_openml_dataset(dataset_id)
    if dataset:
        datasets.append(dataset)
        print(f"  - {dataset['name']}: {dataset['metadata']['n_samples']} muestras, {dataset['metadata']['n_features']} características")

print(f"\nTotal de datasets cargados: {len(datasets)}")


## 3. Extraer Características Meta


In [ ]:
# Extraer características meta de todos los datasets
from meta_features import extract_meta_features_batch

meta_features_df = extract_meta_features_batch(datasets)
meta_features_df


## 4. Visualización de Características Meta


In [ ]:
# Visualizar distribución de características
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

if 'n_samples' in meta_features_df.columns:
    axes[0, 0].hist(meta_features_df['n_samples'], bins=20, edgecolor='black')
    axes[0, 0].set_title('Distribución de Número de Muestras')
    axes[0, 0].set_xlabel('Número de Muestras')
    axes[0, 0].set_ylabel('Frecuencia')

if 'n_features' in meta_features_df.columns:
    axes[0, 1].hist(meta_features_df['n_features'], bins=20, edgecolor='black')
    axes[0, 1].set_title('Distribución de Número de Características')
    axes[0, 1].set_xlabel('Número de Características')
    axes[0, 1].set_ylabel('Frecuencia')

if 'n_classes' in meta_features_df.columns:
    axes[1, 0].bar(meta_features_df['n_classes'].value_counts().index, 
                   meta_features_df['n_classes'].value_counts().values)
    axes[1, 0].set_title('Distribución de Número de Clases')
    axes[1, 0].set_xlabel('Número de Clases')
    axes[1, 0].set_ylabel('Frecuencia')

if 'class_entropy' in meta_features_df.columns:
    axes[1, 1].hist(meta_features_df['class_entropy'], bins=20, edgecolor='black')
    axes[1, 1].set_title('Distribución de Entropía de Clases')
    axes[1, 1].set_xlabel('Entropía')
    axes[1, 1].set_ylabel('Frecuencia')

plt.tight_layout()
plt.show()


## 5. Análisis de Correlaciones


In [ ]:
# Matriz de correlación entre características meta
numeric_cols = meta_features_df.select_dtypes(include=[np.number]).columns
corr_matrix = meta_features_df[numeric_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Matriz de Correlación de Características Meta')
plt.tight_layout()
plt.show()
